# Machine Learning pra Previsão de Evasão Escolar

Neste trabalho será feita uma análise exploratória, limpeza de dados e teste de diversos algoritmos de classificação para prever a evasão escolar.  A acurácia mínima de 90% será buscada.

In [1]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle

## Limpeza e transformação dos dados.
Observar, limpar e transformar os dados.
Essa primeira limpeza deve ser feita para que quando o algoritmo receber as novas informações, ele receba apenas as colunas necessárias com os dados corretos.

In [2]:
# Carrega o arquivo em um dataset.  Apaga a coluna que vem com o dataset.
tecnicoJoinville = pd.read_csv("TECNICOS_Joinville5.csv", encoding = 'latin-1', low_memory = False)
tecnicoJoinville.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [3]:
# Observando as 5 primeiras linhas do dataset
tecnicoJoinville.head()

,produto_educacao,carga_horaria,turno,gratuidade_turma,tipo,online,bairro,cidade,genero,escolaridade,situacao_ocupacional,situacao_matricula,IDADE,empresa
0,MECATRONICA,1600,Noite,Sem_Gratuidade,PF,Nao,ForaJoinville,Guaratuba,M,Medio_C,Empregado,Desistente,28,Britânia
1,ELETROTECNICA,1570,Tarde,Mista,PF,Nao,Costa_e_Silva,Joinville,M,Medio_I,Desempregado,Desistente,17,N_D
2,FABRICACAO,1600,Tarde,Mista,PF,Nao,ForaJoinville,Garuva,M,Medio_I,Desempregado,Desistente,16,N_D
3,INFORMATICA,1600,Tarde,PRONATEC,PF,Nao,Jardim_Sofia,Joinville,M,Medio_I,Desempregado,Desistente,19,N_D
4,AUTOMOTIVO,1600,Tarde,PRONATEC,PF,Nao,Ulysses_Guimaraes,Joinville,M,Medio_I,Desempregado,Desistente,17,N_D


In [4]:
tecnicoJoinville.shape

(1524, 14)

In [5]:
# Verificando se o dataset está desbalanceado
tecnicoJoinville['situacao_matricula'].value_counts()

Matriculado    806
Concluinte     368
Desistente     350
Name: situacao_matricula, dtype: int64

A variável target está desbalanceada. Para que o algoritmo aprenda corretamente é necessário que o dataset esteja balanceado.

## Teste dos algoritmos de classificação

#### Antes de iniciar os testes para escolher o melhor algoritmo, é necessário transformar as variáveis categóricas em variáveis numéricas.
#### Regressão Logística utilizando Ordinal Encoder

In [6]:
# Criando uma lista de nomes dos algoritmos que serão testados. 
# Isso vai permitir criar um dataframe com os resultados de acurácia no final dos testes.
Nomes = ["Regressao", "Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
Acuracia = []

In [7]:
# Pegando apenas os valores, sem o cabeçalho, em formato de array
data = tecnicoJoinville.values

In [8]:
# Visualizando os dados
data

array([['MECATRONICA', '1600', 'Noite', ..., 'Desistente', 28,
        'Britânia'],
       ['ELETROTECNICA', '1570', 'Tarde', ..., 'Desistente', 17, 'N_D'],
       ['FABRICACAO', '1600', 'Tarde', ..., 'Desistente', 16, 'N_D'],
       ...,
       ['ELETROTECNICA', '1570', 'Noite', ..., 'Matriculado', 22, 'N_D'],
       ['MECANICA', '1620', 'Tarde', ..., 'Desistente', 16, 'N_D'],
       ['LOGISTICA', '1300', 'Tarde', ..., 'Desistente', 16, 'N_D']],
      dtype=object)

In [9]:
# Separando a variável target do dataset:
y = data[:,11]

In [10]:
# Tamanho da variável
y.shape

(1524,)

In [11]:
# Pegando todas as outras colunas
X = data[:,[0,1,2,3,4,5,6,7,8,9,10,12,13]]

In [12]:
# Tamanho do array resultante.
X.shape

(1524, 13)

In [13]:
# Visualizando o array
X

array([['MECATRONICA', '1600', 'Noite', ..., 'Empregado', 28, 'Britânia'],
       ['ELETROTECNICA', '1570', 'Tarde', ..., 'Desempregado', 17, 'N_D'],
       ['FABRICACAO', '1600', 'Tarde', ..., 'Desempregado', 16, 'N_D'],
       ...,
       ['ELETROTECNICA', '1570', 'Noite', ..., 'Empregado', 22, 'N_D'],
       ['MECANICA', '1620', 'Tarde', ..., 'Desempregado', 16, 'N_D'],
       ['LOGISTICA', '1300', 'Tarde', ..., 'Desempregado', 16, 'N_D']],
      dtype=object)

In [14]:
# Visualizando a variável target
y

array(['Desistente', 'Desistente', 'Desistente', ..., 'Matriculado',
       'Desistente', 'Desistente'], dtype=object)

In [15]:
# Aplicando Label Encoder aos dados de saída
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [16]:
# Visualizando a variável target de treino após a aplicação do Label Encoder
y

array([1, 1, 1, ..., 2, 1, 1])

### Mean Target Encoder

In [17]:
target_encoder = TargetEncoder()
target_encoder.fit_transform(X=X, y=y)
X = target_encoder.transform(X)

D:\ANACONDA\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [18]:
scaler = StandardScaler(with_mean=False) 
X = scaler.fit_transform(X)
X = scaler.fit_transform(X)

In [19]:
# Separando em conjunto de treino e teste
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.33, random_state = 0)

In [20]:
#Definindo o modelo
modelo5 = LogisticRegression(solver='saga', max_iter=10000)

In [21]:
modelo5.fit(X_train1, y_train1)
# Previsões nos dados de teste
yhat = modelo5.predict(X_test1)
# Avalia a previsão
accuracy = accuracy_score(y_test1, yhat)
print('Accuracy: %.2f' % (accuracy*100))

# Salvando o resultado da regressão logística com Ordinal Encoder
Acuracia.append(accuracy*100)  ##################################  1

Accuracy: 66.20


## Utilizar o K Nearest Neighbors com Mean Target Encoder

In [22]:
# Cria o modelo
modeloK =  KNeighborsClassifier(30)

In [23]:
# Treina o modelo
modeloK.fit(X_train1, y_train1)
# Fazendo previsões
yhat1 = modeloK.predict(X_test1)
score = modeloK.score(X_test1, y_test1)
# Avaliando as previsões
accuracy = accuracy_score(y_test1, yhat1)
print(score*100)

Acuracia.append(score*100) ######################## 2

70.9741550695825


## Utilizar o SVC com  Mean Target Encoder

In [24]:
# Cria o modelo
modeloSVC = SVC(kernel="linear", C=0.025)

# Treina o modelo
modeloSVC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloSVC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)   ######################## 3

71.17296222664015


## Utilizar o SVC com outros parâmetros, Mean Target Encoder

In [25]:
# Cria o modelo
modeloSVC = SVC(gamma=2, C=1)

# Treina o modelo
modeloSVC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloSVC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)   ######################## 4

60.039761431411534


## Utilizar o classificador Gaussiano, Mean Target Encoder

In [26]:
# Cria o modelo
modeloGaus = GaussianProcessClassifier(1.0 * RBF(1.0))

# Treina o modelo
modeloGaus.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloGaus.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

    
Acuracia.append(score*100) ######################## 5

71.57057654075547


## Utilizar o Decision Tree, Mean Target Encoder

In [27]:
# Cria o modelo
modeloDT = DecisionTreeClassifier(max_depth=5)

# Treina o modelo
modeloDT.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloDT.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 6

71.96819085487077


## Utilizar o Random Forest, Mean Target Encoder

In [28]:
# Cria o modelo
modeloRF = RandomForestClassifier(max_depth=50, n_estimators=100, max_features=13)

# Treina o modelo
modeloRF.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloRF.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)
previsoes = modeloRF.predict(X_test1)
Acuracia.append(score*100)  ######################## 7

71.17296222664015


## Utilizar uma rede neural, Mean Target Encoder


In [29]:

# Cria o modelo
modeloMLP = MLPClassifier(alpha=2, max_iter=2000)

# Treina o modelo
modeloMLP.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloMLP.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 8

66.40159045725646


## Utilizar o classificador ADA Boost, Mean Target Encoder

In [30]:
# Cria o modelo
modeloABC =  AdaBoostClassifier()

# Treina o modelo
modeloABC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloABC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 9

63.22067594433399


## Utilizar o classificador Naive Bayes, Mean Target Encoder

In [31]:
# Cria o modelo
modeloGNB =  GaussianNB()

# Treina o modelo
modeloGNB.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloGNB.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 10

53.08151093439364


## Utilizar o classificador Quadratic Discriminant Analysis, Mean Target Encoder

In [32]:
# Cria o modelo
modeloQDA =  QuadraticDiscriminantAnalysis()

# Treina o modelo
modeloQDA.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloQDA.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100) ######################## 11

56.461232604373755


In [33]:
Acuracia

[66.2027833001988,
 70.9741550695825,
 71.17296222664015,
 60.039761431411534,
 71.57057654075547,
 71.96819085487077,
 71.17296222664015,
 66.40159045725646,
 63.22067594433399,
 53.08151093439364,
 56.461232604373755]

In [34]:
#Resultados = pd.DataFrame(Nomes, Acuracia)
Resultados = pd.DataFrame(list(zip(Nomes, Acuracia)),columns =['Nomes', 'Acuracia'])

In [35]:
Resultados

,Nomes,Acuracia
0,Regressao,66.202783
1,Nearest Neighbors,70.974155
2,Linear SVM,71.172962
3,RBF SVM,60.039761
4,Gaussian Process,71.570577
5,Decision Tree,71.968191
6,Random Forest,71.172962
7,Neural Net,66.401590
8,AdaBoost,63.220676
9,Naive Bayes,53.081511


In [36]:

print('Modelo escolhido: %.2f' % (Resultados['Acuracia'].max()))

Modelo escolhido: 71.97


In [37]:
modeloGaus

GaussianProcessClassifier(kernel=1**2 * RBF(length_scale=1))

In [38]:
# Desistente ==> 1
# Concluinte ==> 0
previsoes = modeloGaus.predict(X_test1)
print(previsoes)

[0 2 2 2 2 0 2 2 0 0 2 2 0 2 0 2 1 1 2 2 2 0 2 2 0 2 2 2 2 1 2 2 2 0 2 1 2
 2 2 2 2 2 2 2 2 1 0 2 0 0 2 0 1 2 2 2 0 0 2 1 1 0 2 1 1 2 2 2 0 1 2 0 0 0
 2 0 2 0 0 2 2 1 2 2 2 2 0 0 2 2 2 2 2 1 0 2 0 1 2 0 2 1 2 2 0 1 0 2 0 0 1
 2 2 2 2 0 0 0 2 2 2 1 1 2 2 1 0 0 2 2 2 1 2 0 1 0 1 2 1 2 1 2 2 0 1 2 0 0
 2 1 1 2 0 2 2 2 0 1 2 2 0 2 2 2 2 2 1 1 1 2 2 2 2 2 1 0 0 2 0 2 0 1 1 2 2
 0 0 0 2 1 2 1 2 2 0 2 2 1 0 2 1 2 2 2 2 2 2 2 2 2 2 0 1 1 2 2 2 2 2 2 1 2
 2 2 1 2 2 0 0 1 2 2 2 2 0 2 2 0 2 2 1 0 2 2 1 1 2 2 0 0 2 1 1 1 2 0 2 1 2
 2 1 0 1 2 2 2 2 0 2 2 2 1 0 2 2 2 1 0 0 2 2 0 0 2 1 2 2 2 2 2 0 1 2 2 1 2
 2 2 0 0 2 2 0 1 2 2 2 0 1 2 2 2 2 2 1 2 1 2 0 2 0 2 0 2 2 2 1 2 2 2 2 0 1
 1 2 0 1 2 2 0 2 2 0 1 2 2 1 2 0 0 0 1 2 2 2 2 2 2 2 2 0 2 1 2 1 2 2 2 1 0
 0 1 1 1 2 2 0 1 2 1 0 0 2 2 1 2 2 1 2 2 1 2 0 1 2 2 2 2 0 0 1 1 2 1 2 0 2
 2 1 1 2 2 0 2 2 1 2 0 2 0 2 2 2 2 1 0 2 2 1 0 2 1 2 2 1 2 2 2 2 2 2 0 1 1
 2 2 0 1 0 2 2 2 2 1 2 1 1 2 1 2 2 2 0 1 1 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2 1
 1 2 2 0 2 0 1 2 1 1 1 2 

In [39]:
len(previsoes)

503

In [40]:
respFinal = []
for i in range(0,len(previsoes)):
    if previsoes[i] == 1:
        respFinal.append("Desistente")
    else:
        respFinal.append("Concluinte")
respFinal
    

['Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluin

In [41]:
# save the model to disk
filename = 'ModeloFinal.sav'
pickle.dump(modeloGaus, open(filename, 'wb'))

In [42]:
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [43]:
dados2018 = pd.read_csv("Relatorio_matriculas_Fechado2018.csv", encoding = 'latin-1', low_memory = False)
# Criação de uma lista das colunas utilizadas
colunas = ['cidade_realizacao_turma', 'modalidade', 'produto_educacao', 'carga_horaria', 'inicio', 'turno', 'gratuidade_turma', 'tipo', 'online', 'cpf', 'nascimento', 'bairro', 'cidade', 'genero', 'escolaridade', 'situacao_ocupacional', 'organizacao', 'situacao_matricula']
Limpo2018 = dados2018[colunas]
dadosTotal = pd.DataFrame(Limpo2018)

In [44]:
dadosTotal.dropna(subset=['cpf'], axis = 0, how = 'any', inplace = True)

In [45]:
for i in range(0,len(dadosTotal["produto_educacao"])):
    if dadosTotal.iloc[i,17] == "Evadido / Eliminado" or dadosTotal.iloc[i,17] == "Matriculado / Trancado" or dadosTotal.iloc[i,17] == "Evadido / Desistente" or dadosTotal.iloc[i,17] == "Evadido / Transferido de Curso":
        dadosTotal.iloc[i,17] = "Desistente"
        
for i in range(0,len(dadosTotal["produto_educacao"])):
    if dadosTotal.iloc[i,17] == "Finalizado / Concluinte":
        dadosTotal.iloc[i,17] = "Concluinte"
        
for i in range(0,len(dadosTotal["produto_educacao"])):
    if dadosTotal.iloc[i,17] == "Matriculado / Regular":
        dadosTotal.iloc[i,17] = "Matriculado"